# Jacobian Pseudo Inverse Controller
ToDo: Add theory of operation

## Run In Simulation

In [1]:
import time
import numpy as np
from FR3Py.controllers.waypoint_controller import WaypointController
from FR3Py.envs.pybullet import FR3Sim

robot = FR3Sim(render_mode='human', mode='velocity', Ts=1/1000)
robot.reset()
controller = WaypointController(kp=2)
while True:
    q, dq = robot.get_state()
    dq_cmd = controller.compute(q, dq)
    robot.step(dq_cmd)
    time.sleep(1/1000)

pybullet build time: May 20 2022 19:44:17


interface control mode is: velocity
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Graphics (RKL GT1)
GL_VERSION=4.6 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 21.2.6
Vendor = Intel
Renderer = Mesa Intel(R) Graphics (RKL GT1)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu


KeyboardInterrupt: 